In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
order = pd.read_csv('../input/shopee-code-league-20/_DA_Order_Brushing/order_brush_order.csv')
order.event_time = pd.to_datetime(order.event_time, format ='%Y-%m-%d %H:%M:%S')

df = order.set_index("event_time").sort_index()
process = df.groupby(['shopid', 'userid', pd.Grouper(freq='H', label='left')]).count()

listuserid = []
supspec = process[process.orderid>=3]
supspec.reset_index().groupby('shopid')['userid'].apply(lambda x: listuserid.append(x.values))
def concat_userid(data):
    result = '&'.join(str(x) for x in data)
    return result

bulk_userid = []
for i in listuserid:
    bulk_userid.append(concat_userid(i))

df_brush = pd.DataFrame({"shopid": supspec.reset_index()['shopid'].unique(), "userid": bulk_userid})
df0 = pd.DataFrame({'shopid': df['shopid'].unique(), 'userid': 0})
res_df = pd.concat([df0[~df0.shopid.isin(df_brush.shopid)], df_brush])
res_df.to_csv("submission.csv", index=False)